# Python Question Answering Bot

## Initialization

In [16]:
import pathlib as pth
import sys

sys.path.append(pth.Path.cwd() / "src")

In [17]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from yarl import URL

from wiki_helper.qna.impl.generative_model import LargeLanguageModel
from wiki_helper.qna.impl.knowledge_base import ExternalKnowledgeBase
from wiki_helper.qna.impl.system import RagSystemImpl
from wiki_helper.storing.impl.storage import VectorStorage, VectorStorageConnection

#### Model Locations

In [20]:
generative_model_location = (
    pth.Path.cwd() / "models" / "generative_model" / "Llama-3.2-3B-Instruct-Q6_K.gguf"
)
embedding_model_location = pth.Path.cwd() / "models" / "embedder"

TypeError: Path.absolute() missing 1 required positional argument: 'self'

#### Initialize Embedding Function

In [19]:
prompt_for_retrieval = "Represent this sentence for searching relevant passages:"

contextual_oven = HuggingFaceEmbedding(
    model_name=embedding_model_location.as_posix(),
    query_instruction=prompt_for_retrieval,
)

ValueError: Path /home/z00logist/wiki-helper/examples/models/embedder not found

**NOTE**: you need to up chroma-db before

In [ ]:
settings = VectorStorageConnection(host="localhost", port=8000)

#### Build RAG System

In [ ]:
system = RagSystemImpl(
    LargeLanguageModel(model_location=generative_model_location),
    ExternalKnowledgeBase("en"),
    VectorStorage(embedding_builder=contextual_oven, connection_settings=settings),
)

#### Now We can Train the Model

Let's say we want our system to be able to answer questions about Python

In [ ]:
system.train(
    location=URL("https://en.wikipedia.org/wiki/Python_(programming_language)")
)

Okay, we got the system. We can ask it questions!

In [ ]:
system.answer("What is Python?")